In [ ]:
# Medical Chatbot for Symptom Analysis
# Complete implementation for Google Colab with screenshot-ready interface

# Install required packages
!pip install gradio openai sentence-transformers scikit-learn pandas numpy

import gradio as gr
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Medical Knowledge Base - Symptom to Department Mapping
medical_knowledge_base = {
    "cardiology": {
        "symptoms": [
            "chest pain", "heart palpitations", "shortness of breath during exercise",
            "irregular heartbeat", "chest tightness", "heart racing", "chest pressure",
            "difficulty breathing when lying down", "swelling in legs", "ankle swelling",
            "fatigue during physical activity", "dizziness", "fainting", "rapid heartbeat",
            "slow heartbeat", "chest discomfort", "upper body pain", "arm pain",
            "jaw pain", "neck pain", "back pain between shoulder blades"
        ],
        "description": "Heart and cardiovascular system conditions",
        "common_conditions": ["Heart disease", "Arrhythmia", "High blood pressure", "Heart failure"]
    },
    "dermatology": {
        "symptoms": [
            "skin rash", "itchy skin", "dry skin", "red patches", "skin irritation",
            "acne", "moles", "skin discoloration", "eczema", "hives", "skin bumps",
            "peeling skin", "scaly skin", "skin inflammation", "skin lesions",
            "hair loss", "nail problems", "skin burning", "skin blistering",
            "skin ulcers", "warts", "skin tags", "psoriasis", "dermatitis"
        ],
        "description": "Skin, hair, and nail conditions",
        "common_conditions": ["Eczema", "Psoriasis", "Acne", "Skin cancer", "Dermatitis"]
    },
    "neurology": {
        "symptoms": [
            "headache", "migraine", "dizziness", "seizures", "numbness", "tingling",
            "weakness", "tremor", "coordination problems", "memory loss", "confusion",
            "speech difficulties", "vision problems", "hearing problems", "balance issues",
            "muscle weakness", "muscle stiffness", "difficulty walking", "chronic pain",
            "nerve pain", "burning sensation", "pins and needles", "loss of sensation"
        ],
        "description": "Brain, spinal cord, and nervous system disorders",
        "common_conditions": ["Migraine", "Epilepsy", "Stroke", "Parkinson's disease", "Multiple sclerosis"]
    },
    "gastroenterology": {
        "symptoms": [
            "stomach pain", "abdominal pain", "nausea", "vomiting", "diarrhea",
            "constipation", "bloating", "gas", "heartburn", "acid reflux",
            "difficulty swallowing", "blood in stool", "black stool", "stomach cramps",
            "loss of appetite", "weight loss", "indigestion", "stomach burning",
            "abdominal cramps", "bowel movement changes", "rectal bleeding",
            "stomach discomfort", "intestinal pain"
        ],
        "description": "Digestive system and gastrointestinal conditions",
        "common_conditions": ["GERD", "IBS", "Ulcers", "Crohn's disease", "Hepatitis"]
    },
    "orthopedics": {
        "symptoms": [
            "joint pain", "back pain", "knee pain", "shoulder pain", "hip pain",
            "muscle pain", "bone pain", "stiffness", "swelling in joints",
            "difficulty moving", "limited range of motion", "muscle weakness",
            "joint swelling", "joint stiffness", "bone fracture", "muscle strain",
            "ligament injury", "tendon pain", "arthritis pain", "spine pain",
            "neck pain", "ankle pain", "wrist pain", "elbow pain"
        ],
        "description": "Bones, joints, muscles, and skeletal system",
        "common_conditions": ["Arthritis", "Osteoporosis", "Fractures", "Sports injuries", "Back disorders"]
    },
    "pulmonology": {
        "symptoms": [
            "cough", "shortness of breath", "wheezing", "chest congestion",
            "difficulty breathing", "breathing problems", "chronic cough",
            "coughing up blood", "chest tightness", "respiratory issues",
            "lung pain", "breathing difficulties", "asthma symptoms",
            "bronchitis symptoms", "pneumonia symptoms", "sleep apnea",
            "snoring", "lung congestion", "respiratory distress"
        ],
        "description": "Lungs and respiratory system conditions",
        "common_conditions": ["Asthma", "COPD", "Pneumonia", "Bronchitis", "Sleep apnea"]
    },
    "endocrinology": {
        "symptoms": [
            "diabetes symptoms", "thyroid problems", "hormone imbalance",
            "excessive thirst", "frequent urination", "unexplained weight gain",
            "unexplained weight loss", "fatigue", "mood changes", "hot flashes",
            "cold intolerance", "heat intolerance", "irregular periods",
            "growth problems", "metabolic issues", "blood sugar problems",
            "insulin resistance", "thyroid swelling", "hormone deficiency"
        ],
        "description": "Hormonal and endocrine system disorders",
        "common_conditions": ["Diabetes", "Thyroid disorders", "Hormone imbalances", "Metabolic syndrome"]
    },
    "psychiatry": {
        "symptoms": [
            "depression", "anxiety", "panic attacks", "mood swings", "stress",
            "insomnia", "sleep problems", "mental health issues", "behavioral changes",
            "cognitive problems", "concentration difficulties", "memory issues",
            "emotional distress", "suicidal thoughts", "manic episodes",
            "hallucinations", "delusions", "paranoia", "social withdrawal",
            "substance abuse", "addiction", "eating disorders"
        ],
        "description": "Mental health and psychological conditions",
        "common_conditions": ["Depression", "Anxiety disorders", "Bipolar disorder", "Schizophrenia", "PTSD"]
    },
    "ophthalmology": {
        "symptoms": [
            "vision problems", "eye pain", "blurred vision", "double vision",
            "eye redness", "eye irritation", "dry eyes", "watery eyes",
            "light sensitivity", "night vision problems", "eye strain",
            "floaters", "flashing lights", "loss of vision", "eye discharge",
            "eye swelling", "eyelid problems", "cataracts", "glaucoma symptoms",
            "macular degeneration", "retinal problems"
        ],
        "description": "Eye and vision related conditions",
        "common_conditions": ["Cataracts", "Glaucoma", "Macular degeneration", "Diabetic retinopathy"]
    },
    "ent": {
        "symptoms": [
            "ear pain", "hearing loss", "tinnitus", "ear infection", "throat pain",
            "sore throat", "sinus problems", "nasal congestion", "runny nose",
            "snoring", "voice changes", "difficulty swallowing", "ear discharge",
            "dizziness", "balance problems", "facial pain", "jaw pain",
            "tongue problems", "mouth sores", "bad breath", "gum problems",
            "dental pain", "TMJ symptoms"
        ],
        "description": "Ear, nose, throat, and related conditions",
        "common_conditions": ["Sinusitis", "Hearing loss", "Tinnitus", "Sleep apnea", "Throat infections"]
    }
}

class MedicalChatbot:
    def __init__(self):
        self.vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)
        self.symptoms_data = []
        self.departments = []
        self.conversation_history = []
        self._build_knowledge_base()

    def _build_knowledge_base(self):
        """Build the symptom-department mapping knowledge base"""
        for dept, info in medical_knowledge_base.items():
            for symptom in info["symptoms"]:
                self.symptoms_data.append(symptom)
                self.departments.append(dept)

        # Fit the vectorizer on all symptoms
        self.vectorizer.fit(self.symptoms_data)

    def _extract_symptoms(self, user_input):
        """Extract symptoms from user input using pattern matching"""
        symptoms = []
        user_input_lower = user_input.lower()

        # Common symptom patterns
        patterns = [
            r"i have (.*?)(?:\.|$|,)",
            r"experiencing (.*?)(?:\.|$|,)",
            r"suffering from (.*?)(?:\.|$|,)",
            r"feeling (.*?)(?:\.|$|,)",
            r"my (.*?) (?:hurts|hurt|is|are) (.*?)(?:\.|$|,)",
            r"pain in (.*?)(?:\.|$|,)",
            r"problems with (.*?)(?:\.|$|,)"
        ]

        for pattern in patterns:
            matches = re.findall(pattern, user_input_lower)
            symptoms.extend(matches)

        # Also check for direct symptom mentions
        for symptom in self.symptoms_data:
            if symptom in user_input_lower:
                symptoms.append(symptom)

        return list(set(symptoms))

    def _find_best_department(self, symptoms):
        """Find the best matching department based on symptoms"""
        if not symptoms:
            return None, 0

        # Create a combined symptom string
        symptoms_text = " ".join(symptoms)

        # Vectorize the input symptoms
        input_vector = self.vectorizer.transform([symptoms_text])

        # Calculate similarity with all known symptoms
        symptom_vectors = self.vectorizer.transform(self.symptoms_data)
        similarities = cosine_similarity(input_vector, symptom_vectors)[0]

        # Find department with highest average similarity
        dept_scores = {}
        for i, dept in enumerate(self.departments):
            if dept not in dept_scores:
                dept_scores[dept] = []
            dept_scores[dept].append(similarities[i])

        # Calculate average scores
        avg_scores = {}
        for dept, scores in dept_scores.items():
            avg_scores[dept] = np.mean(scores)

        # Find best match
        best_dept = max(avg_scores, key=avg_scores.get)
        confidence = avg_scores[best_dept]

        return best_dept, confidence

    def _get_follow_up_questions(self, department):
        """Get follow-up questions based on department"""
        questions = {
            "cardiology": [
                "Do you experience chest pain during physical activity?",
                "Have you noticed any irregular heartbeat?",
                "Do you have any family history of heart disease?"
            ],
            "dermatology": [
                "When did you first notice the skin changes?",
                "Is the affected area itchy or painful?",
                "Have you used any new skincare products recently?"
            ],
            "neurology": [
                "How long have you been experiencing these symptoms?",
                "Do the symptoms worsen at any particular time?",
                "Have you had any head injuries recently?"
            ],
            "gastroenterology": [
                "When do you typically experience these symptoms?",
                "Have you noticed any relation to specific foods?",
                "Are you taking any medications currently?"
            ],
            "orthopedics": [
                "Did the pain start after any specific activity or injury?",
                "Does the pain worsen with movement?",
                "Have you had any recent falls or accidents?"
            ],
            "pulmonology": [
                "How long have you had breathing difficulties?",
                "Do you smoke or have you been exposed to dust/chemicals?",
                "Does the cough produce any mucus?"
            ],
            "endocrinology": [
                "Have you noticed any changes in your weight recently?",
                "Are you experiencing increased thirst or urination?",
                "Do you have a family history of diabetes or thyroid issues?"
            ],
            "psychiatry": [
                "How long have you been feeling this way?",
                "Have there been any recent stressful events in your life?",
                "Are you currently taking any medications?"
            ],
            "ophthalmology": [
                "When did you first notice vision changes?",
                "Are you experiencing any eye pain or discomfort?",
                "Do you have diabetes or high blood pressure?"
            ],
            "ent": [
                "How long have you had these symptoms?",
                "Have you had any recent cold or flu?",
                "Do you experience these symptoms seasonally?"
            ]
        }
        return questions.get(department, [])

    def process_message(self, user_input):
        """Process user message and return response"""
        # Add to conversation history
        self.conversation_history.append(f"User: {user_input}")

        # Extract symptoms
        symptoms = self._extract_symptoms(user_input)

        if not symptoms:
            response = """I'd like to help you find the right medical specialist. Could you please describe your symptoms in more detail? For example:

- What specific symptoms are you experiencing?
- When did they start?
- How severe are they?
- Are there any other associated symptoms?

Please describe what you're feeling, and I'll recommend the appropriate medical department to consult."""

        else:
            # Find best department
            best_dept, confidence = self._find_best_department(symptoms)

            if confidence > 0.1:  # Threshold for recommendation
                dept_info = medical_knowledge_base[best_dept]

                response = f"""**Recommended Department: {best_dept.upper()}**

**Specialty:** {dept_info['description']}

**Based on your symptoms:** {', '.join(symptoms)}

**Common conditions treated:**
{', '.join(dept_info['common_conditions'])}

**Follow-up questions to discuss with the specialist:**
"""
                follow_up_questions = self._get_follow_up_questions(best_dept)
                for i, question in enumerate(follow_up_questions, 1):
                    response += f"\n{i}. {question}"

                response += f"""

**Confidence Level:** {confidence:.2%}

**⚠️ Important Disclaimer:** This is an AI-powered recommendation tool for guidance only. Please consult with a healthcare professional for proper medical diagnosis and treatment. In case of emergency, call emergency services immediately.

Would you like to describe any additional symptoms or ask about another condition?"""

            else:
                response = """I couldn't find a strong match for your symptoms with a specific department. This could be because:

1. Your symptoms might be general and could relate to multiple specialties
2. You might need a general practitioner consultation first
3. The symptoms described might need clarification

**I recommend:**
- Start with a **General Practitioner (Family Medicine)** consultation
- They can evaluate your symptoms and refer you to the appropriate specialist if needed

Could you provide more specific details about your symptoms? For example:
- Location of pain or discomfort
- Duration and severity
- Any triggers or patterns you've noticed"""

        # Add response to history
        self.conversation_history.append(f"Bot: {response}")

        return response

    def get_department_info(self, department):
        """Get detailed information about a specific department"""
        if department.lower() in medical_knowledge_base:
            dept_info = medical_knowledge_base[department.lower()]
            return f"""**{department.upper()} Department Information:**

**Specializes in:** {dept_info['description']}

**Common Symptoms Treated:**
{', '.join(dept_info['symptoms'][:10])}...

**Common Conditions:**
{', '.join(dept_info['common_conditions'])}"""
        else:
            return "Department not found. Available departments: " + ", ".join(medical_knowledge_base.keys())

# Initialize the chatbot
chatbot = MedicalChatbot()

# Gradio Interface Functions
def chat_response(message, history):
    """Handle chat responses for Gradio interface"""
    response = chatbot.process_message(message)
    return response

def get_dept_info(department):
    """Get department information"""
    return chatbot.get_department_info(department)

# Create Gradio Interface
def create_interface():
    with gr.Blocks(
        title="Medical Chatbot - Symptom Analysis",
        theme=gr.themes.Soft(),
        css="""
        .gradio-container {
            max-width: 1200px !important;
            margin: auto !important;
        }
        .header {
            text-align: center;
            padding: 20px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border-radius: 10px;
            margin-bottom: 20px;
        }
        """
    ) as iface:

        # Header
        gr.HTML("""
        <div class="header">
            <h1>🏥 Medical Chatbot for Symptom Analysis</h1>
            <p>Get guidance on which medical department to consult based on your symptoms</p>
        </div>
        """)

        with gr.Row():
            with gr.Column(scale=2):
                gr.Markdown("## 💬 Chat with Medical Assistant")

                # Chat Interface
                chatbot_interface = gr.Chatbot(
                    height=400,
                    placeholder="Start by describing your symptoms...",
                    label="Medical Assistant"
                )

                msg = gr.Textbox(
                    placeholder="Describe your symptoms here...",
                    label="Your Message",
                    lines=2
                )

                with gr.Row():
                    submit_btn = gr.Button("Send Message", variant="primary")
                    clear_btn = gr.Button("Clear Chat", variant="secondary")

                # Example buttons
                gr.Markdown("### Quick Examples:")
                with gr.Row():
                    example1 = gr.Button("Chest Pain", size="sm")
                    example2 = gr.Button("Skin Rash", size="sm")
                    example3 = gr.Button("Headache", size="sm")
                    example4 = gr.Button("Stomach Pain", size="sm")

            with gr.Column(scale=1):
                gr.Markdown("## 🏥 Department Information")

                dept_dropdown = gr.Dropdown(
                    choices=list(medical_knowledge_base.keys()),
                    label="Select Department",
                    value="cardiology"
                )

                dept_info_btn = gr.Button("Get Department Info", variant="secondary")

                dept_output = gr.Textbox(
                    label="Department Details",
                    lines=10,
                    interactive=False
                )

                # Statistics
                gr.Markdown("### 📊 System Statistics")
                gr.Markdown(f"""
                - **Departments Available:** {len(medical_knowledge_base)}
                - **Symptoms in Database:** {len(chatbot.symptoms_data)}
                - **AI Model:** TF-IDF + Cosine Similarity
                - **Confidence Threshold:** 10%
                """)

        # Event Handlers
        def respond(message, history):
            if message.strip():
                bot_response = chatbot.process_message(message)
                history.append((message, bot_response))
            return history, ""

        def set_example(example_text):
            return example_text

        # Connect events
        submit_btn.click(respond, [msg, chatbot_interface], [chatbot_interface, msg])
        msg.submit(respond, [msg, chatbot_interface], [chatbot_interface, msg])
        clear_btn.click(lambda: ([], []), outputs=[chatbot_interface])

        # Example buttons
        example1.click(set_example, example1, msg)
        example2.click(set_example, example2, msg)
        example3.click(set_example, example3, msg)
        example4.click(set_example, example4, msg)

        # Department info
        dept_info_btn.click(get_dept_info, dept_dropdown, dept_output)

        # Footer
        gr.Markdown("""
        ---
        **⚠️ Medical Disclaimer:** This chatbot is for informational purposes only and should not replace professional medical advice.
        Always consult with healthcare professionals for proper diagnosis and treatment.

        **🔬 Technology Stack:** Python, scikit-learn, TF-IDF Vectorization, Cosine Similarity, Gradio
        """)

    return iface

# Launch the interface
if __name__ == "__main__":
    print("🚀 Starting Medical Chatbot System...")
    print("📊 Loading knowledge base...")
    print(f"✅ Loaded {len(medical_knowledge_base)} departments")
    print(f"✅ Loaded {len(chatbot.symptoms_data)} symptoms")
    print("🌐 Launching Gradio interface...")

    # Create and launch interface
    interface = create_interface()
    interface.launch(
        share=True,
        server_name="0.0.0.0",
        server_port=7860,
        debug=True,
        show_error=True,
        inbrowser=True
    )

# Test the system programmatically
print("\n" + "="*50)
print("🧪 TESTING MEDICAL CHATBOT SYSTEM")
print("="*50)

# Test cases
test_cases = [
    "I have chest pain and shortness of breath",
    "My skin is itchy and I have red patches",
    "I've been having severe headaches and dizziness",
    "Stomach pain and nausea after eating",
    "Joint pain in my knees and back",
    "Persistent cough and breathing difficulties"
]

print("\n📝 Running test cases...")
for i, test_case in enumerate(test_cases, 1):
    print(f"\n--- Test Case {i} ---")
    print(f"Input: {test_case}")
    response = chatbot.process_message(test_case)
    print(f"Response: {response[:200]}...")
    print("-" * 30)

print("\n✅ All tests completed successfully!")
print("🎯 The chatbot is ready for use!")
print("📸 Take screenshots of the Gradio interface for your demonstration!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2201, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1983, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py", line 631, in postproce